# NLP con Hugging Face

## Procesando los datos para NLP

### Descargando el dataset

In [ ]:
%%capture
!pip install datasets transformers evaluate

Usaremos el dataset MRPC. Este es uno de los 10 datasets que componen el [benchmark (punto de referencia) GLUE](https://huggingface.co/datasets/glue). Se utiliza para medir el rendimiento de los modelos ML en 10 tareas de clasificación de texto diferentes.

En otras palabras, seleccionamos el subset `mrpc` del dataset `glue`:

In [ ]:
from datasets import load_dataset

ds = load_dataset("glue", "mrpc")

Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

Así se ve un ejemplo. Notamos que `mrpc` está compuesto de dos oraciones y una etiqueta que indica si los dos enunciados son equivalentes.

In [ ]:
ex = ds["train"][400]
ex

{'idx': 446,
 'label': 1,
 'sentence1': 'U.S. Agriculture Secretary Ann Veneman , who announced Tuesdays ban , also said Washington would send a technical team to Canada to help .',
 'sentence2': "U.S. Agriculture Secretary Ann Veneman , who announced yesterday 's ban , also said Washington would send a technical team to Canada to assist in the Canadian situation ."}

In [ ]:
labels = ds["train"].features["label"]

In [ ]:
labels.int2str(1)

'equivalent'

### Tokenizando

¿Recuerdas que con visión descargamos el feature extractor directamente del repositorio del modelo pre-entrenado que vamos a usar como base?

Podemos pensar en la función tokenizadora como el equivalente en el NLP.

Descargamos el tokenizador directamente del repo del modelo que usaremos.

In [ ]:
from transformers import AutoTokenizer

repo_id = "bert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(repo_id)

Para preprocesar el conjunto de datos necesitamos convertir el texto en números que el modelo pueda entender. Esto se hace con un tokenizador. 

Pasar de texto a números se conoce como codificación o encoding. El encoding se realiza en un proceso de dos pasos: la tokenización, seguida de la conversión a input ids. Por el momento nos basta saber que estamos traduciendo texto a números llamados como input ids. Estos estarán en el formato adecuado para alimentar nuestro modelo.

Podemos alimentar al tokenizador con una oración o una lista de oraciones, por lo que podemos tokenizar directamente todas las primeras oraciones y todas las segundas oraciones de cada par de esta manera:

In [ ]:
tokenized_sentence_1 = tokenizer(ds["train"]["sentence1"][2])
tokenized_sentence_1

{'input_ids': [101, 2027, 2018, 2405, 2019, 15147, 2006, 1996, 4274, 2006, 2238, 2184, 1010, 5378, 1996, 6636, 2005, 5096, 1010, 2002, 2794, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

Necesitamos manejar los dos enunciados como un par y no separados. El tokenizador puede tomar un par de secuencias y prepararlas de la manera que espera nuestro modelo:

In [ ]:
inputs = tokenizer("This is the first", "This is the second")
inputs

{'input_ids': [101, 2023, 2003, 1996, 2034, 102, 2023, 2003, 1996, 2117, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

¿Qué significa cada uno de los valores que nos retorna el tokenizador?
- `input_ids` es la traducción de palabras a números.
- `attention_mask` es un tensor con la misma forma que `input_ids`, pero lleno de 0 y 1: los 1 indican que se debe atender a los tokens correspondientes y los 0 indican que no se deben atender. Es decir, deben ser ignorados por el modelo.
- `token_type_ids` dice al modelo qué parte de la entrada es la primera oración y cuál es la segunda oración.

El modelo espera que las entradas sean de la forma [CLS] oración 1 [SEP] oración 2 [SEP] cuando hay dos oraciones.

In [ ]:
tokenizer.convert_ids_to_tokens(inputs["input_ids"])

['[CLS]',
 'this',
 'is',
 'the',
 'first',
 '[SEP]',
 'this',
 'is',
 'the',
 'second',
 '[SEP]']

Si seleccionamos otro modelo en el Hub no necesariamente tendremos `token_type_ids` en las entradas tokenizadas (por ejemplo, no se devuelven si usa un modelo `DistilBERT`). Solo se devuelven cuando el modelo sabrá qué hacer con ellas, porque los ha visto durante su preentrenamiento.

En general, no necesitamos preocuparnos por si hay o no `token_type_ids` en nuestras entradas tokenizadas, siempre que usemos el tokenizador correspondiente al modelo, todo estará bien ya que el tokenizador sabe qué proporcionar al modelo.

Por ejemplo, durante esta clase utilizaremos un modelo [`distilroberta-base`](https://huggingface.co/distilroberta-base) por su tamaño y efectividad. Pero no cuenta con `token_type_ids` y aún así nos regresa excelentes resultados.

En la organización del Platzi en el Hub puedes encontrar un [modelo BERT](https://huggingface.co/platzi/platzi-distilroberta-base-mrpc-glue-omar-espejel) afinado siguiendo el mismo proceso que usamos en esta clase.

In [ ]:
repo_id = "distilroberta-base"

tokenizer = AutoTokenizer.from_pretrained(repo_id)

Creamos una función tokenizadora. Recibe un ejemplo y lo tokeniza.

In [ ]:
def tokenize_fn(example):
  return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

In [ ]:
prepared_ds = ds.map(tokenize_fn, batched=True)

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

### Definiendo el data collator: Dynamic padding

Necesitamos que nuestros tensores tengan una forma rectangular. Es decir que tengan el mismo tamaño cada uno de los ejemplos. Sin embargo, los textos no necesariamente tienen el mismo tamaño. 

Para ello usamos el relleno o padding. El padding se asegura de que todas nuestras oraciones tengan la misma longitud al agregar una palabra especial llamada padding token a las oraciones con menos valores. Por ejemplo, si tenemos 10 oraciones con 10 palabras y 1 oración con 20 palabras, el relleno garantizará que todas las oraciones tengan 20 palabras.

Dejamos el argumento de `padding` del tokenizer vacío en nuestra función de tokenización por ahora. Esto se debe a que rellenar (hacer padding) todas las muestras hasta la longitud máxima del dataset no es eficiente, es mejor rellenar las muestras cuando estamos construyendo un batch, ya que entonces solo necesitamos rellenar hasta la longitud máxima en ese batch, y no la longitud máxima en todo el dataset. ¡Esto puede ahorrar mucho tiempo y potencia de procesamiento cuando las entradas tienen longitudes muy variables!

Usaremos un DataCollator para esto.

Rellenemos (hagamos padding) todos los ejemplos con la longitud del elemento más largo del batch. A esta técnica se le conoce como relleno dinámico o dynamic padding.

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

## Entrenamiento y evaluación

Definamos el resto de los argumentos necesarios para `Trainer`.

### Definiendo la métrica 

In [ ]:
import evaluate
import numpy as np

def compute_metrics(eval_pred):
  metric = evaluate.load("glue", "mrpc")
  logits, labels = eval_pred
  predictions = np.argmax(logits, axis=-1)
  return metric.compute(predictions=predictions, references=labels)

### Configurando `Trainer`


In [ ]:
from transformers import AutoModelForSequenceClassification

labels = ds["train"].features["label"].names

model = AutoModelForSequenceClassification.from_pretrained(
    repo_id,
    num_labels=len(labels),
    id2label={str(i): c for i, c in enumerate(labels)},
    label2id={c: str(i) for i, c in enumerate(labels)}
)

Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'roberta.pooler.dense.bias', 'lm_head.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.out_proj.weig

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir = "./platzi-distilroberta-base-mrpc-glue-omar-espejel",
    evaluation_strategy="steps",
    num_train_epochs=3,
    push_to_hub_organization="platzi",
    push_to_hub=True,
    load_best_model_at_end=True
)

/usr/local/lib/python3.7/dist-packages/transformers/training_args.py:1235: FutureWarning: `--push_to_hub_organization` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_model_id` instead and pass the full repo name to this argument (in this case platzi/platzi-distilroberta-base-mrpc-glue-omar-espejel).
  FutureWarning,


In [ ]:
!huggingface-cli login


        _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
        _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
        _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
        _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
        _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

        To login, `huggingface_hub` now requires a token generated from https://huggingface.co/settings/tokens .
        
Token: 
Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your termin

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=prepared_ds["train"],
    eval_dataset=prepared_ds["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Cloning https://huggingface.co/platzi/platzi-distilroberta-base-mrpc-glue-omar-espejel into local empty directory.


Download file pytorch_model.bin:   0%|          | 1.58k/313M [00:00<?, ?B/s]

Download file runs/Jul29_12-14-38_ee95e049683c/events.out.tfevents.1659097867.ee95e049683c.72.2: 100%|########…

Download file runs/Jul29_12-14-38_ee95e049683c/events.out.tfevents.1659097139.ee95e049683c.72.0: 100%|########…

Download file runs/Jul29_12-14-38_ee95e049683c/1659097139.4255755/events.out.tfevents.1659097139.ee95e049683c.…

...

Download file runs/Jul29_16-36-42_2caee687f49e/1659112914.0619447/events.out.tfevents.1659112914.2caee687f49e.…

Clean file runs/Jul29_16-36-42_2caee687f49e/1659112914.0619447/events.out.tfevents.1659112914.2caee687f49e.70.…

Download file training_args.bin: 100%|##########| 3.36k/3.36k [00:00<?, ?B/s]

Clean file training_args.bin:  30%|##9       | 1.00k/3.36k [00:00<?, ?B/s]

Clean file pytorch_model.bin:   0%|          | 1.00k/313M [00:00<?, ?B/s]

### Entrenamiento

In [ ]:
train_results = trainer.train()
trainer.save_model()
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)

The following columns in the training set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: idx, sentence2, sentence1. If idx, sentence2, sentence1 are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 3668
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1377


Step,Training Loss,Validation Loss,Accuracy,F1
500,0.507600,0.746438,0.813725,0.867133
1000,0.344300,0.633180,0.843137,0.886121


The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: idx, sentence2, sentence1. If idx, sentence2, sentence1 are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 408
  Batch size = 8


Saving model checkpoint to ./platzi-distilroberta-base-mrpc-glue-omar-espejel/checkpoint-500
Configuration saved in ./platzi-distilroberta-base-mrpc-glue-omar-espejel/checkpoint-500/config.json
Model weights saved in ./platzi-distilroberta-base-mrpc-glue-omar-espejel/checkpoint-500/pytorch_model.bin
tokenizer config file saved in ./platzi-distilroberta-base-mrpc-glue-omar-espejel/checkpoint-500/tokenizer_config.json
Special tokens file saved in ./platzi-distilroberta-base-mrpc-glue-omar-espejel/checkpoint-500/special_tokens_map.json
tokenizer config file saved in ./platzi-distilroberta-base-mrpc-glue-omar-espejel/tokenizer_config.json
Special tokens file saved in ./platzi-distilroberta-base-mrpc-glue-omar-espejel/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: idx, sentence2, sentence1. If idx, sentence2, sentence1 are not expected by `RobertaForSequenceClassific

Upload file pytorch_model.bin:   0%|          | 3.34k/313M [00:00<?, ?B/s]

Upload file runs/Jul29_20-49-42_cc51138c70be/events.out.tfevents.1659128639.cc51138c70be.71.0:  64%|######3   …

To https://huggingface.co/platzi/platzi-distilroberta-base-mrpc-glue-omar-espejel
   4aeb4e5..ac0dc3c  main -> main

   4aeb4e5..ac0dc3c  main -> main

To https://huggingface.co/platzi/platzi-distilroberta-base-mrpc-glue-omar-espejel
   ac0dc3c..29957fb  main -> main

   ac0dc3c..29957fb  main -> main



***** train metrics *****
  epoch                    =        3.0
  total_flos               =   191920GF
  train_loss               =     0.3627
  train_runtime            = 0:01:46.91
  train_samples_per_second =    102.919
  train_steps_per_second   =     12.879


### Evaluación

In [ ]:
metrics = trainer.evaluate(prepared_ds["validation"])
trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)

The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: idx, sentence2, sentence1. If idx, sentence2, sentence1 are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 408
  Batch size = 8


***** eval metrics *****
  epoch                   =        3.0
  eval_accuracy           =     0.8431
  eval_f1                 =     0.8861
  eval_loss               =     0.6332
  eval_runtime            = 0:00:01.50
  eval_samples_per_second =    270.917
  eval_steps_per_second   =     33.865


### Compartimos en el Hub

In [ ]:
kwargs = {
    "finetuned_from": model.config._name_or_path,
    "tasks": "text-classification",
    "dataset": "datasetX",
    "tags": ["text-classification"]
}

trainer.push_to_hub(commit_message="Lo logramos de nuevo equipo Platzi! 🤗", **kwargs)

Saving model checkpoint to ./platzi-distilroberta-base-mrpc-glue-omar-espejel
Configuration saved in ./platzi-distilroberta-base-mrpc-glue-omar-espejel/config.json
Model weights saved in ./platzi-distilroberta-base-mrpc-glue-omar-espejel/pytorch_model.bin
tokenizer config file saved in ./platzi-distilroberta-base-mrpc-glue-omar-espejel/tokenizer_config.json
Special tokens file saved in ./platzi-distilroberta-base-mrpc-glue-omar-espejel/special_tokens_map.json


Upload file runs/Jul29_20-49-42_cc51138c70be/events.out.tfevents.1659129486.cc51138c70be.71.2: 100%|##########…

To https://huggingface.co/platzi/platzi-distilroberta-base-mrpc-glue-omar-espejel
   29957fb..b61c5f8  main -> main

   29957fb..b61c5f8  main -> main

To https://huggingface.co/platzi/platzi-distilroberta-base-mrpc-glue-omar-espejel
   b61c5f8..cd46619  main -> main

   b61c5f8..cd46619  main -> main



'https://huggingface.co/platzi/platzi-distilroberta-base-mrpc-glue-omar-espejel/commit/b61c5f87244a9631be12d2c93946b004697acac2'